# Training

In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import joblib
import time

# Função para pré-processar imagens
def preprocess_image(image_path, size=(128, 128)):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, size)
    return image

# Função para carregar imagens de pastas organizadas
def load_images_from_folder(folder_path):
    data = []
    labels = []
    class_map = {"bars": 1, "no bars": 0}  # Mapear classes para rótulos
    for class_name, label in class_map.items():
        class_folder = os.path.join(folder_path, class_name)
        for filename in os.listdir(class_folder):
            file_path = os.path.join(class_folder, filename)
            if os.path.isfile(file_path):
                image = preprocess_image(file_path)
                features = hog(image, orientations=9, pixels_per_cell=(8, 8),
                               cells_per_block=(2, 2), visualize=False, channel_axis=None)
                data.append(features)
                labels.append(label)
    return np.array(data), np.array(labels)

# Caminhos das pastas de treino e teste
train_folder = "..."
test_folder ="..."

# Carregar os dados
print("Carregando os dados...")
X_train, y_train = load_images_from_folder(train_folder)
X_test, y_test = load_images_from_folder(test_folder)

# Lista de modelos
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42),
    "k-NN": KNeighborsClassifier(n_neighbors=5, weights='uniform'),
    "Logistic Regression": LogisticRegression(C=1.0, solver='liblinear', random_state=42),
    "Decision Tree": DecisionTreeClassifier(max_depth=None, random_state=42),
    "Naive Bayes": GaussianNB()
}

# Criar pasta para salvar os modelos
output_dir = "..."
os.makedirs(output_dir, exist_ok=True)

# Treinar, avaliar e salvar os modelos
for model_name, model in models.items():
    print(f"\nTreinando e avaliando o modelo: {model_name}")
    start_time = time.time()
    model.fit(X_train, y_train)  # Treinar o modelo
    end_time = time.time()
    
    print(f"Tempo de treinamento: {end_time - start_time:.2f} segundos")
    
    # Fazer predições no conjunto de teste
    y_pred = model.predict(X_test)
    
    # Avaliação
    acc = accuracy_score(y_test, y_pred)
    print(f"Acurácia para {model_name}: {acc:.2f}")
    print(f"Relatório de classificação para {model_name}:")
    print(classification_report(y_test, y_pred))
    
    # Salvar o modelo
    model_path = os.path.join(output_dir, f"{model_name.replace(' ', '_')}.pkl")
    joblib.dump(model, model_path)
    print(f"Modelo {model_name} salvo em: {model_path}")